In [126]:
import numpy as np
import pandas as pd
import asyncio
import aiohttp
import json
import csv

ModuleNotFoundError: No module named 'aiohttp'

In [183]:
import requests
import json
import time
from datetime import datetime
from datetime import date
import numpy as np
import pandas as pd

def get_markets(api = "cryptowatch", active = True):
    """get markets available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            if (active):
                exchanges = {item['exchange'] for item in r if item['active'] is True}
            else:
                exchanges = {item['exchange'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return exchanges

def get_pairs(api = "cryptowatch", active = True):
    """get pairs available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            if (active):
                pairs = {item['pair'] for item in r if item['active'] is True}
            else:
                pairs = {item['pair'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return pairs



In [184]:
print(len(get_pairs()))
print(len(get_pairs(active=False)))
print(len(get_markets()))
print(len(get_markets(active=False)))

1127
1155
25
32


In [170]:
def get_ohlc(market, pair, period = 1440, api = "cryptowatch", start = "", end = "", local_timezone = True):
    """get OHLC prices available in API"""
    period *= 60
    period = str(period)
    params = []
    url = "https://api.cryptowat.ch/markets/" + market + "/" + pair + "/ohlc"
    print(url)
    if (end == ""):
        before = ""
    else:
        before = time.mktime(time.strptime(end, "%d %m %y %H %M"))
    if (start == ""):
        after = start
    else:
        after = time.mktime(time.strptime(start, "%d %m %y %H %M"))
    if (not before or not after):
        prms = [('before', before), ('after', after)]
    else:
        prms = {'before': int(before) , 'after': int(after) }

#    print('params',prms)
    
    if api == "cryptowatch":
        try:
            r = requests.get(url, params=prms).json()['result'][period]
            #ohlc = [item['pair'] for item in r]
            if (local_timezone):            
                a = pd.DataFrame(r, columns = ['date(local)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]] - time.timezone, unit = 's')
                return a
            else:
                a = pd.DataFrame(r, columns = ['date(utc)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')
                return a               
                

        except:
            print("Error: disfunctional API from Cryptowatch")
            
#columns = ['period','open','high','low','close','volume','neg']    

In [171]:
#a = get_ohlc("bitfinex", "btcusd", period = 1, start = time.time()-24*60*60, end = time.time())

s1 = "15 07 18 11 40"
e1 = "15 07 18 11 50"
print(time.timezone)
a = get_ohlc("bitfinex", "btcusd", period = 5, start = s1, end = e1)
b = get_ohlc("bitfinex", "btcusd", period = 5)




10800
https://api.cryptowat.ch/markets/bitfinex/btcusd/ohlc
https://api.cryptowat.ch/markets/bitfinex/btcusd/ohlc


1155
1155


,date(local),open,high,low,close,volume,neg
0,2018-07-15 10:54:00,6352.0,6352.0,6351.9,6352.0,16.447218,104472.63000
1,2018-07-15 10:55:00,6352.0,6352.0,6352.0,6352.0,0.441028,2801.41000
2,2018-07-15 10:56:00,6351.9,6351.9,6351.9,6351.9,0.120406,764.80566
3,2018-07-15 10:57:00,6351.9,6352.0,6351.9,6352.0,0.064003,406.54788
4,2018-07-15 10:58:00,6352.0,6352.0,6351.6,6351.6,5.213264,33114.18000
5,2018-07-15 10:59:00,6351.7,6351.7,6351.6,6351.7,1.844688,11716.79600
6,2018-07-15 11:00:00,6351.6,6351.7,6351.6,6351.6,0.087274,554.32983
7,2018-07-15 11:01:00,6351.7,6351.7,6351.0,6351.0,8.948482,56832.10000
8,2018-07-15 11:02:00,6351.0,6351.1,6351.0,6351.0,2.698791,17140.02300
9,2018-07-15 11:03:00,6351.0,6351.1,6351.0,6351.0,1.350000,8573.85800


# Terça-Feira

- finalizar get_ohlc() (from datetime to timestamp)

- atualizar get_pairs() colocando um parâmetro active = True para retornar só pares ativos

```Python 
pairs = {item['pair'] for item in r if item['active'] is True} ```

- O mesmo para mercados

